# MarkerDB

Original data was pulled from [here](https://markerdb.ca/downloads) (genetics) on 2/14/24. 

In [1]:
import pandas as pd
import re
import table_cleaning_functions as tcf 

## Preliminary Wrangling

In [2]:
# read in raw markerdb data 
markerdb_raw = pd.read_csv('../home/data/raw_data/GlyGen/markerdb_raw.tsv', sep = '\t', header = None)

In [3]:
print(markerdb_raw.shape)
markerdb_raw.head()

(11622, 9)


,0,1,2,3,4,5,6,7,8
0,14030,LRP1,rs1385526,Familial Abdominal Aortic Aneurysm,Pathogenic,NaN,NaN,NaN,NaN
1,14031,ERG,rs2836411,Familial Abdominal Aortic Aneurysm,Pathogenic,NaN,NaN,NaN,NaN
2,14032,ERG,rs602633,Familial Abdominal Aortic Aneurysm,Pathogenic,NaN,NaN,NaN,NaN
3,14035,TBXAS1,rs17837497,Acute Lymphoblastic Leukemia,Pathogenic,NaN,NaN,NaN,NaN
4,14036,TBXAS1,rs17079534,Acute Lymphoblastic Leukemia,Pathogenic,NaN,NaN,NaN,NaN


Add in column headers. 

In [4]:
cols = ['markerdb_id', 'gene', 'rs_id', 'disease', 'other', 'misc', 'misc1', 'misc2', 'misc3']
markerdb_raw.columns = cols
markerdb_raw.head()

,markerdb_id,gene,rs_id,disease,other,misc,misc1,misc2,misc3
0,14030,LRP1,rs1385526,Familial Abdominal Aortic Aneurysm,Pathogenic,NaN,NaN,NaN,NaN
1,14031,ERG,rs2836411,Familial Abdominal Aortic Aneurysm,Pathogenic,NaN,NaN,NaN,NaN
2,14032,ERG,rs602633,Familial Abdominal Aortic Aneurysm,Pathogenic,NaN,NaN,NaN,NaN
3,14035,TBXAS1,rs17837497,Acute Lymphoblastic Leukemia,Pathogenic,NaN,NaN,NaN,NaN
4,14036,TBXAS1,rs17079534,Acute Lymphoblastic Leukemia,Pathogenic,NaN,NaN,NaN,NaN


Check for null values.

In [5]:
print(markerdb_raw.shape)
markerdb_raw.isnull().sum()

(11622, 9)


markerdb_id        0
gene               0
rs_id              0
disease            0
other              2
misc           11622
misc1          11622
misc2          11622
misc3          11622
dtype: int64

Drop unnecessary columns. 

In [6]:
markerdb_raw = markerdb_raw.drop(columns = ['other', 'misc', 'misc1', 'misc2', 'misc3'])
markerdb_raw.head()

,markerdb_id,gene,rs_id,disease
0,14030,LRP1,rs1385526,Familial Abdominal Aortic Aneurysm
1,14031,ERG,rs2836411,Familial Abdominal Aortic Aneurysm
2,14032,ERG,rs602633,Familial Abdominal Aortic Aneurysm
3,14035,TBXAS1,rs17837497,Acute Lymphoblastic Leukemia
4,14036,TBXAS1,rs17079534,Acute Lymphoblastic Leukemia


Only interested in cancer related rows right now, filter on cancer related conditions.

In [7]:
# cancer related words
cancer_related = ['cancer', 'carcinoma', 'leukemia', 'tumor', 'malignancy', 'glioblastoma',
                'melanoma', 'lymphoma', 'sarcoma']

markerdb_raw = markerdb_raw[markerdb_raw['disease'].str.contains('|'.join(cancer_related), case = False, na = False)]
print(markerdb_raw.shape)
markerdb_raw.head()

(9270, 4)


,markerdb_id,gene,rs_id,disease
3,14035,TBXAS1,rs17837497,Acute Lymphoblastic Leukemia
4,14036,TBXAS1,rs17079534,Acute Lymphoblastic Leukemia
5,14037,MAGI2,rs1496766,Acute Lymphoblastic Leukemia
6,14038,PDE4B,rs6683977,Acute Lymphoblastic Leukemia
7,14039,PYGL,rs7142143,Acute Lymphoblastic Leukemia


Recheck for null values.

In [8]:
markerdb_raw.isnull().sum()

markerdb_id    0
gene           0
rs_id          0
disease        0
dtype: int64

Handle other formatting in rs_id column. 

In [9]:
markerdb_raw['rs_id'].value_counts()

rs_id
Arg248Gln ( rs11540652)          7
Arg248Trp ( rs121912651)         7
Arg273His ( rs28934576)          7
Tyr1894Ter ( rs41293497)         7
Trp31Ter ( rs397508045)          6
                                ..
c.787+453del                     1
c.1218_1219insA (p.Ala407fs)     1
c.1211_1212insCT (p.Glu404fs)    1
c.1180_1181insT (p.Gly394fs)     1
c.1107_1108insCT (p.Val370fs)    1
Name: count, Length: 6350, dtype: int64

In [10]:
def extract_rsid(value):
    match = re.search(r'rs\d+', value)
    return match.group(0) if match else None

markerdb_raw['rs_id'] = markerdb_raw['rs_id'].apply(extract_rsid)
markerdb_raw['rs_id'].value_counts(dropna = False)

rs_id
None           4755
rs11540652       13
rs121909229       9
rs41293497        9
rs80358972        7
               ... 
rs554219          1
rs78540526        1
rs2823093         1
rs616488          1
rs4808801         1
Name: count, Length: 2708, dtype: int64

In [11]:
# drop rows with no rs_id
markerdb_raw = markerdb_raw.dropna(subset = ['rs_id'])
markerdb_raw.shape

(4515, 4)

Isolate condition data for cleaning. 

In [90]:
conditions = markerdb_raw['disease']
conditions = conditions.drop_duplicates()
# conditions.to_csv('../home/data/processed_data/GlyGen/markerdb/markerdb_conditions.tsv', sep = '\t', index = False)

In [91]:
markerdb_raw['disease'].value_counts()

disease
Familial Breast Ovarian Cancer                                   2400
Breast Ovarian Cancer                                            1353
Breast Cancer                                                     410
Hereditary Diffuse Gastric Cancer                                  63
PTEN Hamartoma Tumor Syndrome                                      42
Pancreatic Cancer                                                  38
Prostate Cancer                                                    31
Familial Platelet Disorder With Associated Myeloid Malignancy      27
Wilms Tumor 1                                                      20
Somatic Adenocarcinoma of the Lung                                 17
Melanoma                                                           16
Small Cell Cancer Of The Lung                                      10
Ovarian Cancer                                                      9
Acute Myeloid Leukemia                                              9
Bladder Canc

Save processed data and isolate conditions to clean manually. 

In [92]:
markerdb_raw.to_csv('../home/data/processed_data/GlyGen/markerdb/markerdb_processed.tsv', sep = '\t', index = False)

## Secondary Wrangling

In [48]:
markerdb_processed = pd.read_csv('../home/data/processed_data/GlyGen/markerdb/markerdb_processed.tsv', sep = '\t')
markerdb_processed.head()

,markerdb_id,gene,rs_id,disease
0,14035,TBXAS1,rs17837497,Acute Lymphoblastic Leukemia
1,14036,TBXAS1,rs17079534,Acute Lymphoblastic Leukemia
2,14037,MAGI2,rs1496766,Acute Lymphoblastic Leukemia
3,14038,PDE4B,rs6683977,Acute Lymphoblastic Leukemia
4,14039,PYGL,rs7142143,Acute Lymphoblastic Leukemia


In [49]:
markerdb_processed.isnull().sum()

markerdb_id    0
gene           0
rs_id          0
disease        0
dtype: int64

In [50]:
# read in cleaned condition data 
conditions_clean = pd.read_csv('../home/data/processed_data/GlyGen/markerdb/markerdb_conditions.tsv', sep = '\t', dtype = {'DOID': str})
conditions_clean['disease'] = conditions_clean['disease'].str.strip().str.lower()
conditions_clean['condition'] = conditions_clean['condition'].str.strip().str.lower()
conditions_clean.head()

,disease,condition,DOID
0,acute lymphoblastic leukemia,acute lymphoblastic leukemia,:9952
1,basal cell carcinoma,basal cell carcinoma,:2513
2,bladder cancer,bladder carcinoma,:4007
3,breast cancer,breast cancer,:1612
4,cervical cancer,cervical cancer,:4362


In [51]:
df = pd.DataFrame(columns = tcf.TSV_HEADERS)
df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag


Start mapping data to final format.

In [52]:
df['assessed_biomarker_entity'] = markerdb_processed['gene']
df['biomarker'] = 'presence of ' + markerdb_processed['rs_id'].astype(str) + ' mutation in ' + df['assessed_biomarker_entity'].astype(str)
df['assessed_biomarker_entity_id'] = 'dbSNP:' + markerdb_processed['rs_id'].astype(str)
df['assessed_entity_type'] = 'gene'
df['condition'] = markerdb_processed['disease'].str.strip().str.lower()
df['best_biomarker_role'] = 'risk'
df['evidence_source'] = 'MarkerDB:' + markerdb_processed['markerdb_id'].astype(str)
df['tag'] = 'biomarker;assessed_biomarker_entity_id;assessed_biomarker_entity;specimen;condition'

df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
0,NaN,presence of rs17837497 mutation in TBXAS1,TBXAS1,dbSNP:rs17837497,gene,acute lymphoblastic leukemia,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14035,NaN,biomarker;assessed_biomarker_entity_id;assesse...
1,NaN,presence of rs17079534 mutation in TBXAS1,TBXAS1,dbSNP:rs17079534,gene,acute lymphoblastic leukemia,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14036,NaN,biomarker;assessed_biomarker_entity_id;assesse...
2,NaN,presence of rs1496766 mutation in MAGI2,MAGI2,dbSNP:rs1496766,gene,acute lymphoblastic leukemia,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14037,NaN,biomarker;assessed_biomarker_entity_id;assesse...
3,NaN,presence of rs6683977 mutation in PDE4B,PDE4B,dbSNP:rs6683977,gene,acute lymphoblastic leukemia,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14038,NaN,biomarker;assessed_biomarker_entity_id;assesse...
4,NaN,presence of rs7142143 mutation in PYGL,PYGL,dbSNP:rs7142143,gene,acute lymphoblastic leukemia,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14039,NaN,biomarker;assessed_biomarker_entity_id;assesse...


Map condition values.

In [53]:
df_merged = df.merge(conditions_clean, left_on = 'condition', right_on = 'disease', how = 'left')
df_merged.sample(10)

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition_x,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag,disease,condition_y,DOID
4363,NaN,presence of rs572835027 mutation in BRCA1,BRCA1,dbSNP:rs572835027,gene,familial breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:17917,NaN,biomarker;assessed_biomarker_entity_id;assesse...,familial breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
1137,NaN,presence of rs28897759 mutation in BRCA2,BRCA2,dbSNP:rs28897759,gene,breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15596,NaN,biomarker;assessed_biomarker_entity_id;assesse...,breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
241,NaN,presence of rs80357421 mutation in BRCA1,BRCA1,dbSNP:rs80357421,gene,familial breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14562,NaN,biomarker;assessed_biomarker_entity_id;assesse...,familial breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
4195,NaN,presence of rs747773227 mutation in BRCA2,BRCA2,dbSNP:rs747773227,gene,familial breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:17775,NaN,biomarker;assessed_biomarker_entity_id;assesse...,familial breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
1381,NaN,presence of rs28897700 mutation in BRCA2,BRCA2,dbSNP:rs28897700,gene,familial breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15853,NaN,biomarker;assessed_biomarker_entity_id;assesse...,familial breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
1948,NaN,presence of rs1800062 mutation in BRCA1,BRCA1,dbSNP:rs1800062,gene,breast cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:16263,NaN,biomarker;assessed_biomarker_entity_id;assesse...,breast cancer,breast cancer,:1612
2209,NaN,presence of rs1801499 mutation in BRCA2,BRCA2,dbSNP:rs1801499,gene,pancreatic cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:16441,NaN,biomarker;assessed_biomarker_entity_id;assesse...,pancreatic cancer,pancreatic cancer,:1793
1029,NaN,presence of rs55875643 mutation in BRCA2,BRCA2,dbSNP:rs55875643,gene,breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15550,NaN,biomarker;assessed_biomarker_entity_id;assesse...,breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
597,NaN,presence of rs273902792 mutation in BRCA1,BRCA1,dbSNP:rs273902792,gene,familial breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14760,NaN,biomarker;assessed_biomarker_entity_id;assesse...,familial breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683
3145,NaN,presence of rs767165913 mutation in BRCA2,BRCA2,dbSNP:rs767165913,gene,breast ovarian cancer,NaN,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:17022,NaN,biomarker;assessed_biomarker_entity_id;assesse...,breast ovarian cancer,hereditary breast ovarian cancer syndrome,:5683


In [54]:
df['condition'] = df_merged['condition_y'].str.title()
df['condition_id'] = 'DOID' + df_merged['DOID'].astype(str)
df.sample(10)

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
217,NaN,presence of rs397509035 mutation in BRCA1,BRCA1,dbSNP:rs397509035,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14549,NaN,biomarker;assessed_biomarker_entity_id;assesse...
1283,NaN,presence of rs397507571 mutation in BRCA2,BRCA2,dbSNP:rs397507571,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15803,NaN,biomarker;assessed_biomarker_entity_id;assesse...
1040,NaN,presence of rs80358824 mutation in BRCA2,BRCA2,dbSNP:rs80358824,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15556,NaN,biomarker;assessed_biomarker_entity_id;assesse...
1984,NaN,presence of rs80357057 mutation in BRCA1,BRCA1,dbSNP:rs80357057,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:16283,NaN,biomarker;assessed_biomarker_entity_id;assesse...
4417,NaN,presence of rs780485347 mutation in BRCA1,BRCA1,dbSNP:rs780485347,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:17961,NaN,biomarker;assessed_biomarker_entity_id;assesse...
2687,NaN,presence of rs786202043 mutation in BRCA2,BRCA2,dbSNP:rs786202043,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:16734,NaN,biomarker;assessed_biomarker_entity_id;assesse...
3968,NaN,presence of rs1057519992 mutation in TP53,TP53,dbSNP:rs1057519992,gene,Chronic Lymphocytic Leukemia,DOID:1040,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:17602,NaN,biomarker;assessed_biomarker_entity_id;assesse...
3395,NaN,presence of rs876660462 mutation in BRCA2,BRCA2,dbSNP:rs876660462,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:17184,NaN,biomarker;assessed_biomarker_entity_id;assesse...
835,NaN,presence of rs80357018 mutation in BRCA1,BRCA1,dbSNP:rs80357018,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15459,NaN,biomarker;assessed_biomarker_entity_id;assesse...
616,NaN,presence of rs63750843 mutation in MSH2,MSH2,dbSNP:rs63750843,gene,Hereditary Breast Ovarian Cancer Syndrome,DOID:5683,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:15143,NaN,biomarker;assessed_biomarker_entity_id;assesse...


In [55]:
df.shape

(4515, 16)

Drop fully duplicate rows.

In [56]:
df = df.drop_duplicates()
df.shape

(3179, 16)

Assign temporary IDs. 

In [57]:
df.insert(0, 'tmp_id', df.groupby(['biomarker', 'assessed_biomarker_entity_id', 'assessed_biomarker_entity', 'condition']).ngroup())
df.head()

,tmp_id,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
0,366,NaN,presence of rs17837497 mutation in TBXAS1,TBXAS1,dbSNP:rs17837497,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14035,NaN,biomarker;assessed_biomarker_entity_id;assesse...
1,363,NaN,presence of rs17079534 mutation in TBXAS1,TBXAS1,dbSNP:rs17079534,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14036,NaN,biomarker;assessed_biomarker_entity_id;assesse...
2,307,NaN,presence of rs1496766 mutation in MAGI2,MAGI2,dbSNP:rs1496766,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14037,NaN,biomarker;assessed_biomarker_entity_id;assesse...
3,1328,NaN,presence of rs6683977 mutation in PDE4B,PDE4B,dbSNP:rs6683977,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14038,NaN,biomarker;assessed_biomarker_entity_id;assesse...
4,1338,NaN,presence of rs7142143 mutation in PYGL,PYGL,dbSNP:rs7142143,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14039,NaN,biomarker;assessed_biomarker_entity_id;assesse...


In [59]:
# drop biomarker_id column
df.drop(columns = ['biomarker_id'], inplace = True)

# rename tmp_id column to biomarker_id
df.rename(columns = {'tmp_id': 'biomarker_id'}, inplace = True)
df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
0,366,presence of rs17837497 mutation in TBXAS1,TBXAS1,dbSNP:rs17837497,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14035,NaN,biomarker;assessed_biomarker_entity_id;assesse...
1,363,presence of rs17079534 mutation in TBXAS1,TBXAS1,dbSNP:rs17079534,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14036,NaN,biomarker;assessed_biomarker_entity_id;assesse...
2,307,presence of rs1496766 mutation in MAGI2,MAGI2,dbSNP:rs1496766,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14037,NaN,biomarker;assessed_biomarker_entity_id;assesse...
3,1328,presence of rs6683977 mutation in PDE4B,PDE4B,dbSNP:rs6683977,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14038,NaN,biomarker;assessed_biomarker_entity_id;assesse...
4,1338,presence of rs7142143 mutation in PYGL,PYGL,dbSNP:rs7142143,gene,Acute Lymphoblastic Leukemia,DOID:9952,NaN,NaN,risk,NaN,NaN,NaN,MarkerDB:14039,NaN,biomarker;assessed_biomarker_entity_id;assesse...


In [61]:
df['biomarker_id'].value_counts()

biomarker_id
2090    2
2497    2
2015    2
2367    2
2231    2
       ..
2460    1
614     1
613     1
2462    1
2672    1
Name: count, Length: 3110, dtype: int64

In [60]:
# save to tsv
df.to_csv('../home/data/cleaned_data/GlyGen/markerdb.tsv', sep = '\t', index = False)